# This is an example of doing the geocoding we did with OpenRefine but using Python to call the same service

In [1]:
#add your app_id here
APP_ID = '48eab85b'
#add your app_key here
APP_KEY = 'fa31219b4e426ff96a8ded2905e6ae3a'

###We have to tell Python where to find the input file, giving the full path to the file as the first argument and then whether it's reading the file ('r') or writing to the file ('w'). We could also append to a file ('a')
###For more information, see http://www.tutorialspoint.com/python/python_files_io.htm

In [2]:
inputFile = open('Lincoln_Square_BID_Business_List.csv','rU') #change to match the file location
header = next(inputFile) #take the next line of input and assign it to a variable

In [3]:
header #the value of the variable header

'Company,Business Address Street,Business Address City,Business Address State,Business Address Postal Code,Phone Business,Categories\n'

In [4]:
import csv #package for dealing with csv files

###We use the csv.reader function to read in the columns properly, especially the ones that have commas in between double quotation marks.  This is a built in package to Python, but we still have to link to it with the import statement above
###For more information, see https://docs.python.org/2/library/csv.html

In [5]:
inputReader = csv.reader(inputFile) 

In [6]:
#10 lines from the file
for i in range(10):
    print next(inputReader)

['American Apparel ', '1841 Broadway', 'New York', 'NY', '10023-', '(212) 265-3386', 'Business Retail']
['American Bible Society Bookstore', '1865 Broadway', 'New York', 'NY', '10023-', '(212) 408-1385', 'Business Retail']
['American Folk Art Museum Book & Gift Shop', '125 Columbus Avenue', 'New York', 'NY', '10023-', '(212) 595 9533 X121', 'Business Retail']
['Apple Store, Upper West Side', '1981 Broadway', 'New York', 'NY', '10023', '(212) 209-3400', 'Business Retail']
['Banana Republic', '1976 Broadway', 'New York', 'NY', '10023-', '(212) 362-7320', 'Business Retail']
['Barnes & Noble', '1972 Broadway', 'New York', 'NY', '10023', '(212) 595-6859', 'Business Retail']
['BCBG Max Azria', '2005 Broadway', 'New York', 'NY', '10023-', '(212) 496-1853', 'Business Retail']
['Bebe', '10 Columbus Circle', 'New York', 'NY', '10019', '(212) 262-2690', 'Business Retail']
["Bolton's", '181 Amsterdam Avenue', 'New York', 'NY', '10023-', '(212) 362-7396', 'Business Retail']
['Brooks Brothers', '193

In [7]:
test_line = next(inputReader) #the next function returns a line and I'm assigning that line to a variable. 
# This helps me test the input from the text.
test_line

['Burberry Brit',
 '160 Columbus Avenue',
 'New York',
 'NY',
 '10023',
 '(212) 595-0934',
 'Business Retail']

In [9]:
test_line[1]

'New York'

In [11]:
#because we've already iterated through some of the file contents, it's best to re-open the file and start from the beginning
inputFile = open('Lincoln_Square_BID_Business_List.csv','rU') 
header = next(inputFile) #take the next line and assign it to a variable
inputReader = csv.reader(inputFile) 

In [12]:
#this time, we're just printing out the street address
for line in inputReader:
    print line[1]
inputFile.close() #close the open input file

1841 Broadway
1865 Broadway
125 Columbus Avenue
1981 Broadway
1976 Broadway
1972 Broadway
2005 Broadway
10 Columbus Circle
181 Amsterdam Avenue
1934 Broadway
160 Columbus Avenue
201 Amsterdam Avenue
1988 Broadway
1992 Broadway
2007 Broadway
1841 Broadway
154 Columbus Avenue
1885 Broadway
2017 Broadway
1928 Broadway
148 Columbus Avenue
10 Columbus Circle
30 Lincoln Plaza
2030 Broadway
191 Amsterdam Ave.
1965 Broadway
1961 Broadway
151 Amsterdam Avenue
2020 Broadway
1873 Broadway
20 West 64th Street
60 Lincoln Center Plaza
70 Lincoln Center Plaza
2 Columbus Circle
160 Columbus Avenue
1870 Broadway
10 Columbus Circle
10 Columbus Circle
1963 Broadway
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 Columbus Circle
10 

In [20]:
#because we've already iterated through and closed the file, we have to re-open the file
inputFile = open('Lincoln_Square_BID_Business_List.csv','rU') 
header = next(inputFile) #take the next line and assign it to a variable
inputReader = csv.reader(inputFile) 

###This time, we're taking the address and inserting it into the API string I can check the string by clicking the link and it should provide me the response I'm looking for this allows me to test that I've constructed the string properly

In [21]:
for line in inputReader:
    houseNumber = line[1].split(" ")[0] #take the first segment of the address
    street = ' '.join(line[1].split(" ")[1:]) #take the rest of the address as the street
    zipcode = line[4][:5]
    print('https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=' + houseNumber + '&street=' + street + '&zip=' + zipcode + '&app_id=' + APP_ID + '&app_key=' + APP_KEY)

inputFile.close() #close the open input file

https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=1841&street=Broadway&zip=10023&app_id=48eab85b&app_key=fa31219b4e426ff96a8ded2905e6ae3a
https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=1865&street=Broadway&zip=10023&app_id=48eab85b&app_key=fa31219b4e426ff96a8ded2905e6ae3a
https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=125&street=Columbus Avenue&zip=10023&app_id=48eab85b&app_key=fa31219b4e426ff96a8ded2905e6ae3a
https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=1981&street=Broadway&zip=10023&app_id=48eab85b&app_key=fa31219b4e426ff96a8ded2905e6ae3a
https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=1976&street=Broadway&zip=10023&app_id=48eab85b&app_key=fa31219b4e426ff96a8ded2905e6ae3a
https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=1972&street=Broadway&zip=10023&app_id=48eab85b&app_key=fa31219b4e426ff96a8ded2905e6ae3a
https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumb

##Requests package for executing web requests [('HTTP for Humans')](http://docs.python-requests.org/en/latest/)

In [23]:
import requests 

In [27]:
#This is a test call using the values from the last loop using requests.get
r = requests.get('https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=' + houseNumber + '&street=' + street + '&zip=' + zipcode + '&app_id=' + APP_ID + '&app_key=' + APP_KEY)

###Just like with OpenRefine, we have to navigate the tree. The 'u' before the string indicates it's in [Unicode format](http://en.wikipedia.org/wiki/Unicode_symbols)
###I use this test response to know which fields to use in my for loop later on

In [28]:
r.json()

{u'address': {u'assemblyDistrict': u'75',
  u'bbl': u'1011180005',
  u'bblBoroughCode': u'1',
  u'bblTaxBlock': u'01118',
  u'bblTaxLot': u'0005',
  u'boardOfElectionsPreferredLgc': u'1',
  u'boePreferredStreetName': u'WEST   65 STREET',
  u'boePreferredstreetCode': u'13507001',
  u'buildingIdentificationNumber': u'1028160',
  u'businessImprovementDistrict': u'113175',
  u'censusBlock2000': u'1000',
  u'censusBlock2010': u'2000',
  u'censusTract1990': u' 149  ',
  u'censusTract2000': u' 149  ',
  u'censusTract2010': u' 149  ',
  u'cityCouncilDistrict': u'06',
  u'civilCourtDistrict': u'05',
  u'coincidentSegmentCount': u'1',
  u'communityDistrict': u'107',
  u'communityDistrictBoroughCode': u'1',
  u'communityDistrictNumber': u'07',
  u'communitySchoolDistrict': u'03',
  u'condominiumBillingBbl': u'0000000000',
  u'congressionalDistrict': u'10',
  u'cooperativeIdNumber': u'0000',
  u'crossStreetNamesFlagIn': u'E',
  u'dcpCommercialStudyArea': u'11026',
  u'dcpPreferredLgc': u'01',
  u'

In [29]:
r.json()['address']

{u'assemblyDistrict': u'75',
 u'bbl': u'1011180005',
 u'bblBoroughCode': u'1',
 u'bblTaxBlock': u'01118',
 u'bblTaxLot': u'0005',
 u'boardOfElectionsPreferredLgc': u'1',
 u'boePreferredStreetName': u'WEST   65 STREET',
 u'boePreferredstreetCode': u'13507001',
 u'buildingIdentificationNumber': u'1028160',
 u'businessImprovementDistrict': u'113175',
 u'censusBlock2000': u'1000',
 u'censusBlock2010': u'2000',
 u'censusTract1990': u' 149  ',
 u'censusTract2000': u' 149  ',
 u'censusTract2010': u' 149  ',
 u'cityCouncilDistrict': u'06',
 u'civilCourtDistrict': u'05',
 u'coincidentSegmentCount': u'1',
 u'communityDistrict': u'107',
 u'communityDistrictBoroughCode': u'1',
 u'communityDistrictNumber': u'07',
 u'communitySchoolDistrict': u'03',
 u'condominiumBillingBbl': u'0000000000',
 u'congressionalDistrict': u'10',
 u'cooperativeIdNumber': u'0000',
 u'crossStreetNamesFlagIn': u'E',
 u'dcpCommercialStudyArea': u'11026',
 u'dcpPreferredLgc': u'01',
 u'dotStreetLightContractorArea': u'1',
 u'd

In [31]:
r.json()['address']['latitude'], r.json()['address']['longitude']

(40.77228046241657, -73.98036321722904)

In [22]:
#because we've already iterated through and closed the file, we have to re-open the file
inputFile = open('Lincoln_Square_BID_Business_List.csv','rU') 
header = next(inputFile) #take the next line and assign it to a variable
inputReader = csv.reader(inputFile) 

In [35]:
#This time, I want to process the response I get. I'm going to get the response and write the output to a file
#Here I'm opening the file I'm going to write to. If the file already exists, it overwrite the file.
outputFile = open('Lincoln_Square_BID_Business_List_geocoded.csv','w') #opens the file
outputFile.write(header.rstrip() + ',longitude,latitude\n') #writes the first line of my CSV file. NOTE: '\n' writes a new line

###This time through we put the pieces together.  I'm executing the web request, I'm getting the reponse, and then writing it to the output

In [36]:
for line in inputReader:
    houseNumber = line[1].split(" ")[0] #take the first segment of the address
    street = ' '.join(line[1].split(" ")[1:]) #take the rest of the address as the street
    zipcode = line[4][:5]
    #the web request, using the get() function from the package requests and assigning it to the variable r
    r = requests.get('https://api.cityofnewyork.us/geoclient/v1/address.json?houseNumber=' + houseNumber + '&street=' + street + '&zip=' + zipcode + '&app_id=' + APP_ID + '&app_key=' + APP_KEY)
    
    
    #My default value is 'Null' for longitude. 0 is a legitimate value and when I map these, I don't want anything to plot to 0,0
    longitude = 'Null' 
    latitude = 'Null'

    
    #longitude
    if 'longitude' in r.json()['address'].keys(): #I first check that there is a longitude present
        #if it's present, I assign the value to the variable "longitude". Otherwise, the variable longitude has the value 'Null'
        longitude = r.json()['address']['longitude'] 
    #latitude
    if 'latitude' in r.json()['address'].keys(): #I then check that there is a latitude present
        #if it's present, I assign the value to the variable "latitude". Otherwise, the variable latitude has the value 'Null'
        latitude = r.json()['address']['latitude']
    #I then write the result to the output file, stringing together the variables
    #because the latitude and longitude are decimal numbers, I have to convert them to strings using the str() function
    #in order to combine them together with the rest of the variables. 
    #Note the commas in the string and the '\n' new line character at the end of the string
    outputFile.write(','.join(line) + ',' + str(longitude) + ',' + str(latitude) + '\n')

inputFile.close() #close the input file
outputFile.close() #close the open output file. 
#This is important because if you don't close it, it's possible the last bit of data won't get written to the file before the 
#program closes.

##And we're done